In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('C:/Users/sam79/OneDrive/桌面/W210 Yelp/data'):
    
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 

# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

C:/Users/sam79/OneDrive/桌面/W210 Yelp/data\yelp_academic_dataset_business.json
C:/Users/sam79/OneDrive/桌面/W210 Yelp/data\yelp_academic_dataset_checkin.json
C:/Users/sam79/OneDrive/桌面/W210 Yelp/data\yelp_academic_dataset_review.json
C:/Users/sam79/OneDrive/桌面/W210 Yelp/data\yelp_academic_dataset_tip.json
C:/Users/sam79/OneDrive/桌面/W210 Yelp/data\yelp_academic_dataset_user.json
C:/Users/sam79/OneDrive/桌面/W210 Yelp/data\yelp_dataset.tar
C:/Users/sam79/OneDrive/桌面/W210 Yelp/data\yelp_dataset\yelp_dataset


<a id="data"></a>
# Data

We are using subsets of each table since we have a large dataset to work with. For this notebook, we used _business_ and _review_ tables.

In [34]:
# import the data (chunksize returns jsonReader for iteration)
businesses = pd.read_json("C:/Users/sam79/OneDrive/桌面/W210 Yelp/data/yelp_academic_dataset_business.json", lines=True, orient='columns', chunksize=1000000)
reviews = pd.read_json("C:/Users/sam79/OneDrive/桌面/W210 Yelp/data/yelp_academic_dataset_review.json", lines=True, orient='columns', chunksize=1000000)

In [35]:
# read the data 
for business in businesses:
    subset_business = business
    break
    
for review in reviews:
    subset_review = review
    break

In [4]:
# peak the tables
display(subset_business.head(2))
display(subset_review.head(2))

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."


,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18


In [5]:
print(subset_business.shape)
print(subset_review.shape)

(150346, 14)
(1000000, 9)


In [6]:
# No New York, Maybe in other Naming convention ?
subset_business[(subset_business['city'].str.contains('New'))]['city'].value_counts()

New Orleans         6209
New Port Richey      604
Newark               359
Newtown              322
New Hope             246
New Castle           217
Newtown Square       176
New Britain           17
Newport               14
New Palestine         13
New Whiteland          6
Newfield               6
New Washoe City        4
New orleans            2
New Tampa              1
Newtown Sq.            1
Newtown Sqaure         1
Newcastle              1
Newtown Sq             1
New Britian            1
Newportville           1
Newtown Township       1
Newton Square          1
New Orlaens            1
Newtown square         1
New Berlinville        1
New Pt Richey          1
New Jersey             1
New Prt Rchy           1
New castle             1
Newton                 1
Name: city, dtype: int64

In [7]:
# No San Diego and San Francisco, Maybe in other Naming convention ?
subset_business[(subset_business['city'].str.contains('San'))]['city'].value_counts()

Santa Barbara                       3829
San Antonio                           14
Santa Barbara                          5
Sanatoga                               3
Santa Barbra                           2
Santa Ynez                             1
Santa Clara                            1
Santa Rita                             1
San Anselmo                            1
Santa Barbara,                         1
Santa Barbara & Ventura Counties       1
Santa  Barbara                         1
Santa Maria                            1
Name: city, dtype: int64

In [8]:
# No Paris, Maybe in other Naming convention ?
subset_business[(subset_business['city'].str.contains('Paris'))]['city'].value_counts()

Jefferson Parish    1
Name: city, dtype: int64

In [9]:
subset_business['city'].value_counts().sort_values(ascending=False)

Philadelphia    14569
Tucson           9250
Tampa            9050
Indianapolis     7540
Nashville        6971
                ...  
Spark               1
Sullivan            1
St Boise            1
Merion              1
Apollo beach        1
Name: city, Length: 1416, dtype: int64

<a id="preprocessing-data"></a>
# Preprocessing the Data

We chose Philadelphia since it has the highest number of restraunts. The restaurant is the most popular category among businesses. 

In [10]:
# Businesses in Philadelphia and currently open business
city = subset_business[(subset_business['city'].str.contains('Philadelphia')) & (subset_business['is_open'] == 1)]
Philadelphia = city[['business_id','name','address', 'categories', 'attributes','stars']]
Philadelphia

,business_id,name,address,categories,attributes,stars
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'RestaurantsDelivery': 'False', 'OutdoorSeati...",4.0
15,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,205 Race St,"Sushi Bars, Restaurants, Japanese","{'RestaurantsReservations': 'True', 'Restauran...",4.0
19,ROeacJQwBeh05Rqg7F6TCg,BAP,1224 South St,"Korean, Restaurants","{'NoiseLevel': 'u'quiet'', 'GoodForMeal': '{'d...",4.5
35,aPNXGTDkf-4bjhyMBQxqpQ,Craft Hall,901 N Delaware Ave,"Eatertainment, Arts & Entertainment, Brewpubs,...","{'OutdoorSeating': 'True', 'RestaurantsPriceRa...",3.5
82,ppFCk9aQkM338Rgwpl2F5A,Wawa,3604 Chestnut St,"Restaurants, Automotive, Delis, Gas Stations, ...","{'Alcohol': 'u'none'', 'RestaurantsGoodForGrou...",3.0
...,...,...,...,...,...,...
150306,wVxXRFf10zTTAs11nr4xeA,PrimoHoagies,6024 Ridge Ave,"Restaurants, Specialty Food, Food, Sandwiches,...","{'NoiseLevel': 'u'average'', 'RestaurantsTakeO...",3.0
150326,JDM7kiCEmeBc_5W-z6eZZw,2601 Parkway Apartments,2601 Pennsylvania Ave,"Real Estate, Apartments, Home Services",{'BusinessAcceptsCreditCards': 'True'},3.0
150329,9U1Igcpe954LoWZRmNc-zg,Hand & Stone Massage And Facial Spa,"1100 S Columbus Blvd, Ste 24","Day Spas, Beauty & Spas, Skin Care, Massage","{'BusinessAcceptsCreditCards': 'True', 'Wheelc...",3.0
150334,LJ4GjQ1HL6kqvIPpNUNNaQ,Shanti Yoga and Ayurveda,"1638 Pine St, Fl 1","Health & Medical, Yoga, Shopping, Naturopathic...","{'ByAppointmentOnly': 'True', 'GoodForKids': '...",4.5


In [11]:
# getting just restaurants from Philadelphia business
rest = Philadelphia[Philadelphia['categories'].str.contains('Restaurant.*')==True].reset_index()
rest

,index,business_id,name,address,categories,attributes,stars
0,3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'RestaurantsDelivery': 'False', 'OutdoorSeati...",4.0
1,15,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,205 Race St,"Sushi Bars, Restaurants, Japanese","{'RestaurantsReservations': 'True', 'Restauran...",4.0
2,19,ROeacJQwBeh05Rqg7F6TCg,BAP,1224 South St,"Korean, Restaurants","{'NoiseLevel': 'u'quiet'', 'GoodForMeal': '{'d...",4.5
3,35,aPNXGTDkf-4bjhyMBQxqpQ,Craft Hall,901 N Delaware Ave,"Eatertainment, Arts & Entertainment, Brewpubs,...","{'OutdoorSeating': 'True', 'RestaurantsPriceRa...",3.5
4,82,ppFCk9aQkM338Rgwpl2F5A,Wawa,3604 Chestnut St,"Restaurants, Automotive, Delis, Gas Stations, ...","{'Alcohol': 'u'none'', 'RestaurantsGoodForGrou...",3.0
...,...,...,...,...,...,...,...
3526,150178,auwFZzfhe2pvFw43OfsAfw,Stina Pizzeria,1705 Snyder Ave,"Pizza, Restaurants, Mediterranean","{'DogsAllowed': 'False', 'RestaurantsTakeOut':...",4.5
3527,150210,K1SsvIPfFcHniNSPc3IG7g,Flip-N-Pizza,1308 W Girard Ave,"Restaurants, American (Traditional), Chicken W...","{'WiFi': 'u'no'', 'BYOB': 'False', 'Restaurant...",4.0
3528,150237,OfRbGvHn-nAnQMqNxiIpcA,Grilly Cheese,3306 Arch St,"Specialty Food, Event Planning & Services, Res...","{'BikeParking': 'True', 'WheelchairAccessible'...",3.0
3529,150306,wVxXRFf10zTTAs11nr4xeA,PrimoHoagies,6024 Ridge Ave,"Restaurants, Specialty Food, Food, Sandwiches,...","{'NoiseLevel': 'u'average'', 'RestaurantsTakeO...",3.0


<a id="get-dummies"></a>
* ** Get Dummies from attributes and categories columns**

> In "attributes" column has nested attributes. In order to create a feature table, we need to separate those nested attributes into their own columns. Therefore, the following functions will be used to achieve this goal.

In [12]:
# Function that extract keys from the nested dictionary
def extract_keys(attr, key):
    if attr == None:
        return "{}"
    if key in attr:
        return attr.pop(key)

# convert string to dictionary
import ast
def str_to_dict(attr):
    if attr != None:
        return ast.literal_eval(attr)
    else:
        return ast.literal_eval("{}")    

In [13]:
# get dummies from nested attributes
rest['BusinessParking'] = rest.apply(lambda x: str_to_dict(extract_keys(x['attributes'], 'BusinessParking')), axis=1)
rest['Ambience'] = rest.apply(lambda x: str_to_dict(extract_keys(x['attributes'], 'Ambience')), axis=1)
rest['GoodForMeal'] = rest.apply(lambda x: str_to_dict(extract_keys(x['attributes'], 'GoodForMeal')), axis=1)
rest['Dietary'] = rest.apply(lambda x: str_to_dict(extract_keys(x['attributes'], 'Dietary')), axis=1)
rest['Music'] = rest.apply(lambda x: str_to_dict(extract_keys(x['attributes'], 'Music')), axis=1)

In [14]:
rest

,index,business_id,name,address,categories,attributes,stars,BusinessParking,Ambience,GoodForMeal,Dietary,Music
0,3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,"Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'RestaurantsDelivery': 'False', 'OutdoorSeati...",4.0,"{'garage': False, 'street': True, 'validated':...",{},{},{},{}
1,15,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,205 Race St,"Sushi Bars, Restaurants, Japanese","{'RestaurantsReservations': 'True', 'Restauran...",4.0,"{'valet': False, 'garage': None, 'street': Tru...","{'touristy': False, 'hipster': False, 'romanti...","{'dessert': True, 'latenight': None, 'lunch': ...",{},{}
2,19,ROeacJQwBeh05Rqg7F6TCg,BAP,1224 South St,"Korean, Restaurants","{'NoiseLevel': 'u'quiet'', 'RestaurantsGoodFor...",4.5,"{'garage': False, 'street': True, 'validated':...","{'touristy': False, 'hipster': False, 'romanti...","{'dessert': False, 'latenight': False, 'lunch'...",{},{}
3,35,aPNXGTDkf-4bjhyMBQxqpQ,Craft Hall,901 N Delaware Ave,"Eatertainment, Arts & Entertainment, Brewpubs,...","{'OutdoorSeating': 'True', 'RestaurantsPriceRa...",3.5,"{'garage': False, 'street': False, 'validated'...","{'touristy': False, 'hipster': False, 'romanti...","{'dessert': False, 'latenight': False, 'lunch'...",{},{}
4,82,ppFCk9aQkM338Rgwpl2F5A,Wawa,3604 Chestnut St,"Restaurants, Automotive, Delis, Gas Stations, ...","{'Alcohol': 'u'none'', 'RestaurantsGoodForGrou...",3.0,"{'garage': False, 'street': True, 'validated':...","{'romantic': False, 'intimate': False, 'classy...",{},{},{}
...,...,...,...,...,...,...,...,...,...,...,...,...
3526,150178,auwFZzfhe2pvFw43OfsAfw,Stina Pizzeria,1705 Snyder Ave,"Pizza, Restaurants, Mediterranean","{'DogsAllowed': 'False', 'RestaurantsTakeOut':...",4.5,"{'valet': False, 'garage': None, 'street': Tru...","{'divey': False, 'hipster': None, 'casual': Tr...","{'dessert': None, 'latenight': None, 'lunch': ...",{},{}
3527,150210,K1SsvIPfFcHniNSPc3IG7g,Flip-N-Pizza,1308 W Girard Ave,"Restaurants, American (Traditional), Chicken W...","{'WiFi': 'u'no'', 'BYOB': 'False', 'Restaurant...",4.0,{},{},{},{},{}
3528,150237,OfRbGvHn-nAnQMqNxiIpcA,Grilly Cheese,3306 Arch St,"Specialty Food, Event Planning & Services, Res...","{'BikeParking': 'True', 'WheelchairAccessible'...",3.0,{},{},{},{},{}
3529,150306,wVxXRFf10zTTAs11nr4xeA,PrimoHoagies,6024 Ridge Ave,"Restaurants, Specialty Food, Food, Sandwiches,...","{'NoiseLevel': 'u'average'', 'RestaurantsTakeO...",3.0,"{'garage': False, 'street': True, 'validated':...","{'touristy': None, 'hipster': False, 'romantic...","{'dessert': False, 'latenight': False, 'lunch'...",{},{}


In [15]:
# create table with attribute dummies
df_attr = pd.concat([ rest['attributes'].apply(pd.Series), rest['BusinessParking'].apply(pd.Series),
                    rest['Ambience'].apply(pd.Series), rest['GoodForMeal'].apply(pd.Series), 
                    rest['Dietary'].apply(pd.Series) ], axis=1)
df_attr_dummies = pd.get_dummies(df_attr)
df_attr_dummies

,RestaurantsDelivery_False,RestaurantsDelivery_None,RestaurantsDelivery_True,OutdoorSeating_False,OutdoorSeating_None,OutdoorSeating_True,BusinessAcceptsCreditCards_False,BusinessAcceptsCreditCards_None,BusinessAcceptsCreditCards_True,BikeParking_False,...,latenight_False,latenight_True,lunch_False,lunch_True,dinner_False,dinner_True,brunch_False,brunch_True,breakfast_False,breakfast_True
0,1,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,1,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
2,0,1,0,0,1,0,0,0,1,0,...,1,0,0,1,0,1,1,0,1,0
3,0,0,1,0,0,1,0,0,1,0,...,1,0,1,0,1,0,1,0,1,0
4,0,0,1,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3526,0,0,1,0,0,1,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3527,0,0,1,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3528,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3529,0,0,1,1,0,0,0,0,1,0,...,1,0,0,1,0,1,1,0,1,0


In [16]:
# get dummies from categories
df_categories_dummies = pd.Series(rest['categories']).str.get_dummies(',')
df_categories_dummies

,Acai Bowls,Active Life,Adult,Adult Entertainment,Afghan,African,Airport Lounges,Airport Shuttles,American (New),American (Traditional),...,Trinidadian,Turkish,Uzbek,Vegan,Vegetarian,Venues & Event Spaces,Vietnamese,Wine Bars,Wineries,Wraps
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3526,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3527,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3528,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3529,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
# pull out names and stars from rest table 
result = rest[['name','stars']]
result

,name,stars
0,St Honore Pastries,4.0
1,Tuna Bar,4.0
2,BAP,4.5
3,Craft Hall,3.5
4,Wawa,3.0
...,...,...
3526,Stina Pizzeria,4.5
3527,Flip-N-Pizza,4.0
3528,Grilly Cheese,3.0
3529,PrimoHoagies,3.0


In [18]:
# Concat all tables and drop Restaurant column
df_final = pd.concat([df_attr_dummies, df_categories_dummies, result], axis=1)
df_final.drop('Restaurants',inplace=True,axis=1)

In [19]:
# map floating point stars to an integer
mapper = {1.0:1,1.5:2, 2.0:2, 2.5:3, 3.0:3, 3.5:4, 4.0:4, 4.5:5, 5.0:5}
df_final['stars'] = df_final['stars'].map(mapper)

In [20]:
# Final table for the models 
df_final

,RestaurantsDelivery_False,RestaurantsDelivery_None,RestaurantsDelivery_True,OutdoorSeating_False,OutdoorSeating_None,OutdoorSeating_True,BusinessAcceptsCreditCards_False,BusinessAcceptsCreditCards_None,BusinessAcceptsCreditCards_True,BikeParking_False,...,Uzbek,Vegan,Vegetarian,Venues & Event Spaces,Vietnamese,Wine Bars,Wineries,Wraps,name,stars
0,1,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,St Honore Pastries,4
1,0,0,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,Tuna Bar,4
2,0,1,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,BAP,5
3,0,0,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,Craft Hall,4
4,0,0,1,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,Wawa,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3526,0,0,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,Stina Pizzeria,5
3527,0,0,1,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,Flip-N-Pizza,4
3528,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,Grilly Cheese,3
3529,0,0,1,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,PrimoHoagies,3


## Check how many attributes(Tags) in the dataset for restraunts for our recomendation algorithms

In [21]:
# Check how many attributes(Tags) for restraunts for our recomendation algorithms
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
df_final.head()

,RestaurantsDelivery_False,RestaurantsDelivery_None,RestaurantsDelivery_True,OutdoorSeating_False,OutdoorSeating_None,OutdoorSeating_True,BusinessAcceptsCreditCards_False,BusinessAcceptsCreditCards_None,BusinessAcceptsCreditCards_True,BikeParking_False,BikeParking_None,BikeParking_True,RestaurantsPriceRange2_1,RestaurantsPriceRange2_2,RestaurantsPriceRange2_3,RestaurantsPriceRange2_4,RestaurantsPriceRange2_None,RestaurantsTakeOut_False,RestaurantsTakeOut_None,RestaurantsTakeOut_True,ByAppointmentOnly_False,ByAppointmentOnly_True,WiFi_'free',WiFi_'no',WiFi_'paid',WiFi_None,WiFi_u'free',WiFi_u'no',WiFi_u'paid',Alcohol_'beer_and_wine',Alcohol_'full_bar',Alcohol_'none',Alcohol_None,Alcohol_u'beer_and_wine',Alcohol_u'full_bar',Alcohol_u'none',Caters_False,Caters_None,Caters_True,RestaurantsReservations_False,RestaurantsReservations_None,RestaurantsReservations_True,RestaurantsGoodForGroups_False,RestaurantsGoodForGroups_None,RestaurantsGoodForGroups_True,RestaurantsAttire_'casual',RestaurantsAttire_'dressy',RestaurantsAttire_'formal',RestaurantsAttire_None,RestaurantsAttire_u'casual',RestaurantsAttire_u'dressy',RestaurantsAttire_u'formal',HasTV_False,HasTV_None,HasTV_True,GoodForKids_False,GoodForKids_None,GoodForKids_True,NoiseLevel_'average',NoiseLevel_'loud',NoiseLevel_'quiet',NoiseLevel_'very_loud',NoiseLevel_None,NoiseLevel_u'average',NoiseLevel_u'loud',NoiseLevel_u'quiet',NoiseLevel_u'very_loud',DogsAllowed_False,DogsAllowed_None,DogsAllowed_True,HappyHour_False,HappyHour_True,WheelchairAccessible_False,WheelchairAccessible_True,RestaurantsTableService_False,RestaurantsTableService_True,BusinessAcceptsBitcoin_False,BusinessAcceptsBitcoin_True,BYOBCorkage_'no',BYOBCorkage_'yes_corkage',BYOBCorkage_'yes_free',BYOBCorkage_None,BYOBCorkage_u'no',BYOBCorkage_u'yes_corkage',BYOBCorkage_u'yes_free',Corkage_False,Corkage_None,Corkage_True,GoodForDancing_False,GoodForDancing_True,BestNights_None,"BestNights_{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': False, 'saturday': False}","BestNights_{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': False, 'saturday': True}","BestNights_{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': True, 'saturday': False}","BestNights_{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': True, 'saturday': True}","BestNights_{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': True, 'sunday': False, 'saturday': False}","BestNights_{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': True, 'sunday': False, 'saturday': True}","BestNights_{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': True, 'sunday': True, 'saturday': True}","BestNights_{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': True, 'thursday': False, 'sunday': False, 'saturday': False}","BestNights_{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': True, 'thursday': False, 'sunday': False, 'saturday': True}","BestNights_{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': True, 'thursday': False, 'sunday': True, 'saturday': False}","BestNights_{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': True, 'thursday': True, 'sunday': False, 'saturday': True}","BestNights_{'monday': False, 'tuesday': False, 'friday': True, 'wednesday': False, 'thursday': False, 'sunday': False, 'saturday': False}","BestNights_{'monday': False, 'tuesday': False, 'friday': True, 'wednesday': False, 'thursday': False, 'sunday': False, 'saturday': True}","BestNights_{'monday': False, 'tuesday': False, 'friday': True, 'wednesday': False, 'thursday': False, 'sunday': True, 'saturday': True}","BestNights_{'monday': False, 'tuesday': False, 'friday': True, 'wednesday': False, 'thursday': True, 'sunday': False, 'saturday'

In [22]:
# List out all attributes with values
df_final.drop(['name','stars'], axis =1).sum(axis=0).sort_values(ascending = False).head(100)

RestaurantsTakeOut_True             3110
valet_False                         2900
BusinessAcceptsCreditCards_True     2765
validated_False                     2732
garage_False                        2532
 Restaurants                        2512
RestaurantsDelivery_True            2476
upscale_False                       2473
touristy_False                      2442
romantic_False                      2405
lot_False                           2328
intimate_False                      2300
divey_False                         2280
hipster_False                       2250
HasTV_True                          2122
trendy_False                        2107
RestaurantsGoodForGroups_True       1996
GoodForKids_True                    1949
BikeParking_True                    1888
RestaurantsReservations_False       1874
classy_False                        1814
street_True                         1799
dessert_False                       1696
breakfast_False                     1630
OutdoorSeating_F

<a id="content-based"></a>
# Content Based Filtering- Model

In this section, we are going to build a system that recognizes the similarity between restaurants based on specific features and recommends restaurants that are most similar to a particular restaurant. __df_final__ (features) table used to build this system.

In [23]:
# Create X (all the features) and y (target)
X = df_final.iloc[:,:-2]
y = df_final['stars']

* **Split the data into train and test set (80:20)**

In [24]:
# Split the data into train and test sets
from sklearn.model_selection import train_test_split
X_train_knn, X_test_knn, y_train_knn, y_test_knn = train_test_split(X, y, test_size=0.2, random_state=1)

* **Instantiate and fit the model**

In [25]:
y_train_knn.head()

3277    5
273     4
3224    3
558     5
3487    3
Name: stars, dtype: int64

In [26]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

knn = KNeighborsClassifier(n_neighbors=20)
knn.fit(X_train_knn, y_train_knn)

#y_pred = knn.predict(X_test)

accuracy_train = knn.score(X_train_knn, y_train_knn)
accuracy_test = knn.score(X_test_knn, y_test_knn)

print(f"Score on training set: {accuracy_train}")
print(f"Score on test set: {accuracy_test}")

Score on training set: 0.5513456090651558
Score on test set: 0.4738330975954738


C:\Users\sam79\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\sam79\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim

The restaurant of the validation set

In [27]:
# look at the last row for the test
display(df_final.iloc[-1:])

# look at the restaurant name from the last row.
print("Validation set (Restaurant name): ", df_final['name'].values[-1])

,RestaurantsDelivery_False,RestaurantsDelivery_None,RestaurantsDelivery_True,OutdoorSeating_False,OutdoorSeating_None,OutdoorSeating_True,BusinessAcceptsCreditCards_False,BusinessAcceptsCreditCards_None,BusinessAcceptsCreditCards_True,BikeParking_False,BikeParking_None,BikeParking_True,RestaurantsPriceRange2_1,RestaurantsPriceRange2_2,RestaurantsPriceRange2_3,RestaurantsPriceRange2_4,RestaurantsPriceRange2_None,RestaurantsTakeOut_False,RestaurantsTakeOut_None,RestaurantsTakeOut_True,ByAppointmentOnly_False,ByAppointmentOnly_True,WiFi_'free',WiFi_'no',WiFi_'paid',WiFi_None,WiFi_u'free',WiFi_u'no',WiFi_u'paid',Alcohol_'beer_and_wine',Alcohol_'full_bar',Alcohol_'none',Alcohol_None,Alcohol_u'beer_and_wine',Alcohol_u'full_bar',Alcohol_u'none',Caters_False,Caters_None,Caters_True,RestaurantsReservations_False,RestaurantsReservations_None,RestaurantsReservations_True,RestaurantsGoodForGroups_False,RestaurantsGoodForGroups_None,RestaurantsGoodForGroups_True,RestaurantsAttire_'casual',RestaurantsAttire_'dressy',RestaurantsAttire_'formal',RestaurantsAttire_None,RestaurantsAttire_u'casual',RestaurantsAttire_u'dressy',RestaurantsAttire_u'formal',HasTV_False,HasTV_None,HasTV_True,GoodForKids_False,GoodForKids_None,GoodForKids_True,NoiseLevel_'average',NoiseLevel_'loud',NoiseLevel_'quiet',NoiseLevel_'very_loud',NoiseLevel_None,NoiseLevel_u'average',NoiseLevel_u'loud',NoiseLevel_u'quiet',NoiseLevel_u'very_loud',DogsAllowed_False,DogsAllowed_None,DogsAllowed_True,HappyHour_False,HappyHour_True,WheelchairAccessible_False,WheelchairAccessible_True,RestaurantsTableService_False,RestaurantsTableService_True,BusinessAcceptsBitcoin_False,BusinessAcceptsBitcoin_True,BYOBCorkage_'no',BYOBCorkage_'yes_corkage',BYOBCorkage_'yes_free',BYOBCorkage_None,BYOBCorkage_u'no',BYOBCorkage_u'yes_corkage',BYOBCorkage_u'yes_free',Corkage_False,Corkage_None,Corkage_True,GoodForDancing_False,GoodForDancing_True,BestNights_None,"BestNights_{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': False, 'saturday': False}","BestNights_{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': False, 'saturday': True}","BestNights_{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': True, 'saturday': False}","BestNights_{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': False, 'sunday': True, 'saturday': True}","BestNights_{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': True, 'sunday': False, 'saturday': False}","BestNights_{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': True, 'sunday': False, 'saturday': True}","BestNights_{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': False, 'thursday': True, 'sunday': True, 'saturday': True}","BestNights_{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': True, 'thursday': False, 'sunday': False, 'saturday': False}","BestNights_{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': True, 'thursday': False, 'sunday': False, 'saturday': True}","BestNights_{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': True, 'thursday': False, 'sunday': True, 'saturday': False}","BestNights_{'monday': False, 'tuesday': False, 'friday': False, 'wednesday': True, 'thursday': True, 'sunday': False, 'saturday': True}","BestNights_{'monday': False, 'tuesday': False, 'friday': True, 'wednesday': False, 'thursday': False, 'sunday': False, 'saturday': False}","BestNights_{'monday': False, 'tuesday': False, 'friday': True, 'wednesday': False, 'thursday': False, 'sunday': False, 'saturday': True}","BestNights_{'monday': False, 'tuesday': False, 'friday': True, 'wednesday': False, 'thursday': False, 'sunday': True, 'saturday': True}","BestNights_{'monday': False, 'tuesday': False, 'friday': True, 'wednesday': False, 'thursday': True, 'sunday': False, 'saturday'

Validation set (Restaurant name):  Adelita Taqueria & Restaurant


* **Test the model:** 

> We used the last row as a validation set (we didn't include this last row for modeling). 

In [28]:
# test set from the df_final table (only last row): Restaurant name: "Steak & Cheese & Quick Pita Restaurant"
test_set = df_final.iloc[-1:,:-2]

# validation set from the df_final table (exclude the last row)
X_val =  df_final.iloc[:-1,:-2]
y_val = df_final['stars'].iloc[:-1]

In [29]:
# fit model with validation set
n_knn = knn.fit(X_val, y_val)

After fitting the KNN model to the validation set, we are going to find the distances between the validation set and the other restaurants based on their similar features. 

In [30]:
# distances and indeces from validation set (Steak & Cheese & Quick Pita Restaurant)
distances, indeces =  n_knn.kneighbors(test_set)
#n_knn.kneighbors(test_set)[1][0]

# create table distances and indeces from "Steak & Cheese & Quick Pita Restaurant"
final_table = pd.DataFrame(n_knn.kneighbors(test_set)[0][0], columns = ['distance'])
final_table['index'] = n_knn.kneighbors(test_set)[1][0]
final_table.set_index('index')

,distance
index,
3119,4.000000
3098,4.123106
1785,4.123106
1195,4.242641
930,4.358899
3443,4.358899
1983,4.358899
1050,4.358899
586,4.472136


We are creating the following ***result*** table which displays similar restaurants to the validation restrauant by their distances. Based on this recommendation system, the short distance means having more similarity to the validation restrauant.

In [31]:
# get names of the restaurant that similar to the validation restrauant
result = final_table.join(df_final,on='index')
result[['distance','index','name','stars']].head(5)

,distance,index,name,stars
0,4.000000,3119,Los Taquitos de Puebla,4
1,4.123106,3098,Yummy Sushi,4
2,4.123106,1785,The Pizza Place,3
3,4.242641,1195,The Flavor Spot,4
4,4.358899,930,Artigiano Pizza,3
